In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

data = pd.read_csv('combined_metrics.csv')

# Preprocessing

# Get the shape of the data
(N, P) = data.shape

# Split data into input features X and true output Y
X = data.values[:, :-1]  # All columns except the last one
Y = data.values[:, -1]   # The last column

if np.any(np.isinf(X)):
    print("Input contains infinite values. Replacing with nan.")
    max_float64 = np.finfo(np.float64).max
    X[np.isinf(X)] = max_float64

# One-hot encode the output labels
Y = np.array(
    [[1, 0, 0, 0, 0] if (t == 1) else 
     [0, 1, 0, 0, 0] if (t == 2) else 
     [0, 0, 1, 0, 0] if (t == 3) else 
     [0, 0, 0, 1, 0] if (t == 4) else 
     [0, 0, 0, 0, 1] for t in Y]
)

# Different architectures
hidden_layer_sizes = [
    (7, 9, 7),        # Architecture 1
    (8, 16, 8),       # Architecture 2
    (16, 32, 16),     # Architecture 3
    (32, 64, 32),     # Architecture 4
    (64, 128, 64)     # Architecture 5
]

# Parameters for neural network
input_neurons = X.shape[1]
output_neurons = 5
learning_rate = 0.001
epochs = 1000

# Store results for plotting
results = []

# K-Fold Cross Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Define sigmoid and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Define softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Subtracting max value for numerical stability
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

for hidden_neurons_1, hidden_neurons_2, hidden_neurons_3 in hidden_layer_sizes:
    accuracies = []
    for train_index, test_index in kf.split(X):
        # Split data into training and test sets for this fold
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # Normalize the data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize random weights and biases
        np.random.seed(42)
        W1 = np.random.rand(input_neurons, hidden_neurons_1)
        b1 = np.random.rand(1, hidden_neurons_1)
        W2 = np.random.rand(hidden_neurons_1, hidden_neurons_2)
        b2 = np.random.rand(1, hidden_neurons_2)
        W3 = np.random.rand(hidden_neurons_2, hidden_neurons_3)
        b3 = np.random.rand(1, hidden_neurons_3)
        W4 = np.random.rand(hidden_neurons_3, output_neurons)
        b4 = np.random.rand(1, output_neurons)

        # Training loop
        for epoch in range(epochs):
            # Forward propagation
            Z1 = np.dot(X_train, W1) + b1
            H1 = relu(Z1)
            Z2 = np.dot(H1, W2) + b2
            H2 = relu(Z2)
            Z3 = np.dot(H2, W3) + b3
            H3 = relu(Z3)
            Z4 = np.dot(H3, W4) + b4
            Y_pred = relu(Z4)

            # Compute loss
            loss = np.mean(np.square(Y_pred - Y_train))

            # Backpropagation
            dZ4 = (Y_pred - Y_train) * relu_derivative(Y_pred)
            dW4 = np.dot(H3.T, dZ4)
            db4 = np.sum(dZ4, axis=0, keepdims=True)
            dZ3 = np.dot(dZ4, W4.T) * relu_derivative(H3)
            dW3 = np.dot(H2.T, dZ3)
            db3 = np.sum(dZ3, axis=0, keepdims=True)
            dZ2 = np.dot(dZ3, W3.T) * relu_derivative(H2)
            dW2 = np.dot(H1.T, dZ2)
            db2 = np.sum(dZ2, axis=0, keepdims=True)
            dZ1 = np.dot(dZ2, W2.T) * relu_derivative(H1)
            dW1 = np.dot(X_train.T, dZ1)
            db1 = np.sum(dZ1, axis=0, keepdims=True)

            # Update weights and biases
            W4 -= learning_rate * dW4
            b4 -= learning_rate * db4
            W3 -= learning_rate * dW3
            b3 -= learning_rate * db3
            W2 -= learning_rate * dW2
            b2 -= learning_rate * db2
            W1 -= learning_rate * dW1
            b1 -= learning_rate * db1

            # Print the loss every 100 epochs
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

        # Evaluate the model on the test set for this fold
        Z1 = np.dot(X_test, W1) + b1
        H1 = relu(Z1)
        Z2 = np.dot(H1, W2) + b2
        H2 = relu(Z2)
        Z3 = np.dot(H2, W3) + b3
        H3 = relu(Z3)
        Z4 = np.dot(H3, W4) + b4
        Y_pred = relu(Z4)
        Y_pred = softmax(Y_pred)

        # Calculate accuracy
        arr_x = np.argmax(Y_pred, axis=1)
        arr_y = np.argmax(Y_test, axis=1)
        correct = np.sum(arr_x == arr_y)
        accuracy = correct / len(arr_x)
        accuracies.append(accuracy)
        print(f'Fold accuracy: {accuracy}')

    # Calculate the average accuracy across all folds
    average_accuracy = np.mean(accuracies)
    print(f'Average accuracy for hidden layers {hidden_neurons_1}, {hidden_neurons_2}, and {hidden_neurons_3}: {average_accuracy}')
    results.append((hidden_neurons_1, hidden_neurons_2, hidden_neurons_3, average_accuracy))

# Plotting the results
hidden_layer_descriptions = [f'{h1}-{h2}-{h3}' for h1, h2, h3 in hidden_layer_sizes]
accuracies = [result[3] for result in results]

plt.figure(figsize=(10, 6))
plt.bar(hidden_layer_descriptions, accuracies, color='skyblue')
plt.xlabel('Hidden Layer Sizes')
plt.ylabel('Average Accuracy')
plt.title('10-Fold Cross-Validation Accuracy for Different Hidden Layer Sizes')
plt.ylim([0, 1])
plt.show()


Epoch 0, Loss: nan
Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan
Fold accuracy: 0.2255125284738041
Epoch 0, Loss: nan
Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan
Fold accuracy: 0.2255125284738041
Epoch 0, Loss: nan
Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan


KeyboardInterrupt: 